In [2]:
from scipy.spatial import KDTree
import numpy as np

In [39]:
N_points = 1000000

data = np.random.random([N_points,3])
data

array([[0.89955413, 0.71025835, 0.40496701],
       [0.40332481, 0.70713442, 0.72629897],
       [0.76350004, 0.73577755, 0.20103106],
       ...,
       [0.40183904, 0.36298547, 0.35907315],
       [0.55471504, 0.24653043, 0.47355998],
       [0.11874531, 0.20025302, 0.53890354]])

In [34]:
data_KDT = KDTree(data)
data_KDT.data

array([[0.09001691, 0.55263371, 0.09001691],
       [0.69279939, 0.97724461, 0.69279939],
       [0.02435897, 0.01150908, 0.02435897],
       ...,
       [0.66633455, 0.65058214, 0.66633455],
       [0.95384502, 0.1975908 , 0.95384502],
       [0.85749241, 0.9793585 , 0.85749241]])

In [35]:
data_KDT.query_ball_point([0.53,0.247,0.5301], 0.08)

[264449,
 928975,
 754499,
 505316,
 112379,
 764351,
 329453,
 774564,
 177619,
 810000,
 947905,
 33753,
 952775,
 565985,
 809132,
 54223,
 849404,
 975708,
 571057,
 281394,
 143445,
 618026,
 248501,
 703457,
 310656,
 235303,
 29224,
 487930,
 334963,
 54500,
 710008,
 733955,
 862156,
 587889,
 996005,
 666491,
 909730,
 474902,
 381566,
 356581,
 636645,
 355142,
 417994,
 799514,
 610477,
 177235,
 770273,
 721457,
 442912,
 562550,
 288835,
 138686,
 962050,
 999969,
 384535,
 373955,
 934200,
 766364,
 141166,
 490657,
 583221,
 299351,
 661469,
 875545,
 568086,
 771096,
 563331,
 417689,
 914735,
 284349,
 963149,
 91226,
 231049,
 750856,
 651951,
 781614,
 388413,
 709236,
 830566,
 148026,
 987525,
 330277,
 110402,
 170384,
 554326,
 205297,
 786675,
 827783,
 732307,
 802252,
 529001,
 409304,
 409703,
 212201,
 543957,
 488925,
 542834,
 702125,
 779288,
 747479,
 637110,
 336140,
 741392,
 835203,
 315801,
 45154,
 338268,
 596021,
 684942,
 242072,
 774846,
 474446